In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')

#  Objective and Theory¶

The main idea in this note book will be to first calculate the CPM which is the calculated based on the revenue generaed by the publisher(which is 1% of the actual revenue) thus calculating the CPM.

Once the CPM is calculated keeping that as a target variable we will try to create a model which will try to predict the reverse price ( We will take CPM roughly as the reverse price because any advertisers will try to reduce the cost and increase the value which they generate from Inventory and without the reverse price it will be most often [Rationally] lower than the actual Intrinsic value to the advertisers.)

Now once we have the reverse price algorithm we know that any advertisers will be willing to go around 20% above the value in order to secure the slot if he sees value in the slot.

# Data descriptive analysis and Preprocessing¶

Removing the transaction id related data

In [ ]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

In [ ]:
df.drop(['order_id' , 'line_item_type_id'], axis = 1, inplace=True)

In [ ]:
df.head()

In [ ]:
corr = df.corr()
plt.figure(figsize=(12,8))
sns.heatmap(data=corr,vmin=0, vmax=1, cmap="YlGnBu",  square=True)
plt.show()

In [ ]:
df['total_revenue'].sum()

In [ ]:
#calculating CPM
#calculating the value that the Advertisers Bid for the month of June
# CPM(the value which was the winning bid value) = 
#((revenue of the publisher*100)/revenue_share_percentage)/measurable_impressions)*1000

def weird_division(n, d):
    return n / d if d else 0

df['CPM'] = df.apply(lambda x: weird_division(((x['total_revenue']*100)),x['measurable_impressions'])*1000 , axis=1)


In [ ]:
#we can remove integration type as it has only one value and revenue share percent as that we have already used and 
#is only one single value as well
df.drop(['integration_type_id' , 'revenue_share_percent'], axis = 1, inplace=True)

In [ ]:
corr = df.corr()
plt.figure(figsize=(14,10))
sns.heatmap(data=corr,vmin=0, vmax=1, cmap="YlGnBu",  square=True, annot= True)
plt.show()

In [ ]:
# we can remove total impressions as well as that is account the same information as measurable impressions 
# also let us try to see if viewable/measurable impressions are corellated to revenue or not 

df['View/measurable'] = df.apply(lambda x: weird_division(x['viewable_impressions'],x['measurable_impressions']) , axis=1)


In [ ]:
df.drop([ 'total_impressions'], axis = 1, inplace=True)

In [ ]:
corr = df.corr()
plt.figure(figsize=(14,10))
sns.heatmap(data=corr,vmin=0, vmax=1, cmap="YlGnBu",  square=True, annot= True)
plt.show()

In [ ]:
df.nunique()

In [ ]:
df.head()

In [ ]:
df.fillna(0,  inplace = True)

In [ ]:
df.isnull().sum()

In [ ]:
corr = df.corr()
plt.figure(figsize=(14,10))
sns.heatmap(data=corr,vmin=0, vmax=1, cmap="YlGnBu",  square=True, annot= True)
plt.show()

In [ ]:
df_numericals = df[['total_revenue','viewable_impressions','measurable_impressions','CPM','View/measurable']]

In [ ]:
df_numericals.describe()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df_numericals)
scaled_data = scaler.transform(df_numericals)

In [ ]:
corr = df_numericals.corr()
plt.figure(figsize=(14,10))
sns.heatmap(data=corr,vmin=0, vmax=1, cmap="YlGnBu",  square=True, annot= True)
plt.show()

In [ ]:
sns.boxplot(df['total_revenue'] )

In [ ]:
sns.boxplot(df['CPM'] )

In [ ]:
df[df['total_revenue']<0].head(10)
#we have a value which has a negative revenue not sure what it means or if it is even valid ! removing some outliers 

In [ ]:
#remove the outliers 

df = df[df['CPM'].between(df['CPM'].quantile(.05), df['CPM'].quantile(.95))]

In [ ]:
df.reset_index(inplace= True)

In [ ]:
df[df['total_revenue']<0].head(10)
#we had a value which has a negative revenue which is removed and extreme values are removed as well 

In [ ]:
corr = df.corr()
plt.figure(figsize=(14,10))
sns.heatmap(data=corr,vmin=0, vmax=1, cmap="YlGnBu",  square=True, annot= True)
plt.show()

# Creating new dummy variables by one hot encoding and then PCA

In [ ]:
monetization_channel_list = pd.get_dummies(df.monetization_channel_id, prefix='monetization_channel_', drop_first=True)


In [ ]:
from sklearn.decomposition import PCA
pca2 = PCA(n_components=2)
pca2.fit(monetization_channel_list)
X_transformed_mone_list = pd.DataFrame(pca2.transform(monetization_channel_list))

variance = pca2.explained_variance_ratio_ #calculate variance ratios

var=np.cumsum(np.round(pca2.explained_variance_ratio_, decimals=3)*100)
var
#with just 2 PC we are able to get 95% variance explained so let us add the PC and remove the column

In [ ]:
df.shape

In [ ]:
#with just 2 PC we are able to get 95% variance explained so let us add the PC and remove the column
X_transformed_mone_list = X_transformed_mone_list.add_prefix('Monetization_')
df = pd.concat([df, X_transformed_mone_list], axis=1)

In [ ]:
df.columns

In [ ]:
X_transformed_mone_list.shape

In [ ]:
#performing one hot encoding 

device_category_list = pd.get_dummies(df.device_category_id, prefix='Ad_type', drop_first=True)

In [ ]:
# performing PCA on top of one hot encoding 

pca2 = PCA(n_components=2)
pca2.fit(device_category_list)
X_transformed_devise = pd.DataFrame(pca2.transform(device_category_list))

variance = pca2.explained_variance_ratio_ #calculate variance ratios

var=np.cumsum(np.round(pca2.explained_variance_ratio_, decimals=3)*100)
var
#with just two PC we are able to get 99% variance explained so let us add the PC and remove the column

In [ ]:
#with just two PC we are able to get 99% variance explained so let us add the PC and remove the column

X_transformed_devise = X_transformed_devise.add_prefix('Devise_')
df = pd.concat([df, X_transformed_devise], axis=1)

SITE

In [ ]:
# performing one hot encoding

site_list = pd.get_dummies(df.site_id, prefix='Site', drop_first=True)

In [ ]:
#Checking if PCA helps in reduction of the number of features (If we are able to explain above 95% variance we will go ahead)

pca2 = PCA(n_components=7)
pca2.fit(site_list)
X_transformed_site = pd.DataFrame(pca2.transform(site_list))

variance = pca2.explained_variance_ratio_ #calculate variance ratios

var=np.cumsum(np.round(pca2.explained_variance_ratio_, decimals=3)*100)
var

In [ ]:
#only able to reduce 3 features so let us leave all the columns as is as one hot encoded
df = pd.concat([df, site_list], axis=1)

ADVERTISERS

In [ ]:
advertiser_list = pd.get_dummies(df.advertiser_id, prefix='advertiser_', drop_first=True)

In [ ]:
pca2 = PCA(n_components=10)
pca2.fit(advertiser_list)
X_transformed_adver = pd.DataFrame(pca2.transform(advertiser_list))

variance = pca2.explained_variance_ratio_ #calculate variance ratios

var=np.cumsum(np.round(pca2.explained_variance_ratio_, decimals=3)*100)
var
#with just 10 PC we are able to get 99% variance explained so let us add the PC and remove the column

In [ ]:
#with just 10 PC we are able to get 99% variance explained so let us add the PC and remove the column
X_transformed_adver = X_transformed_adver.add_prefix('Advert_')
df = pd.concat([df, X_transformed_adver], axis=1)

In [ ]:
df.describe()

In [ ]:
df.columns

Ad units

I am not taking the approach of doing a PCA around Ad units as i tried that already and was not able to reduce it by much and explain the variance till 95% thus what i am doing now is - ranking the units based on the amount of CPM they corrospond to and create an ordinal ranking to make the algorithm's job easier rather than giving more that 90 PCA componenets or more than 100 one hot encoded features which reduced the algorithm's performace which i observed.

Same approach is taken for GEO ID as well

In [ ]:
df_ad_unit = df.groupby(['ad_unit_id'])['CPM'].agg(['sum', 'idxmax'])
df_ad_unit['sum'] = df_ad_unit['sum'].astype('int64') 
df_ad_unit.sort_values(by='sum',ascending=False, inplace = True)
#only certain advertisers give a lot of money 

In [ ]:
len(df_ad_unit)

In [ ]:
df_ad_unit['ad_unit_id'] = df_ad_unit.index
df_ad_unit.index = range(0, 132)
df_ad_unit['ad_unit_rank'] =  range(0, 132)
df_ad_unit.head()

In [ ]:
df = pd.merge(df,
                 df_ad_unit[['ad_unit_rank', 'ad_unit_id']],
                 on='ad_unit_id') 

In [ ]:
df.head()

In [ ]:
df_geo = df.groupby(['geo_id'])['CPM'].agg(['sum', 'idxmax'])
df_geo['sum'] = df_geo['sum'].astype('int64') 
df_geo.sort_values(by='sum',ascending=False, inplace = True)
#only certain advertisers give a lot of money 

In [ ]:
len(df_geo)
df_geo['geo_id'] = df_geo.index
df_geo.index = range(0, 219)
df_geo['geo_rank'] =  range(0, 219)
df_geo.head()

In [ ]:
df = pd.merge(df,
                 df_geo[['geo_rank', 'geo_id']],
                 on='geo_id') 

In [ ]:
sns.lineplot(data=df_geo['sum'])

In [ ]:
df_date = df.groupby(['date'])['CPM'].agg(['sum', 'idxmax'])
df_date['sum'] = df_date['sum'].astype('int64') 
df_date.head(30)

In [ ]:
plt.figure(figsize=(14,10))
sns.lineplot(data = df_date, x=df_date.index, y = 'sum' )

plt.xticks(rotation=70)
plt.show()

# Modeling

In [ ]:
df.columns

In [ ]:
df2 = df[df['total_revenue'] !=0.0 ]

In [ ]:
from sklearn.model_selection import train_test_split
X = df[['View/measurable','measurable_impressions', 'viewable_impressions','ad_type_id','Monetization_0', 'Monetization_1', 'Advert_0', 'Advert_1', 'Advert_2', 'Advert_3', 'Advert_4', 'Advert_5',
       'Advert_6', 'Advert_7', 'Advert_8', 'Advert_9', 'Devise_0', 'Devise_1', 'geo_rank', 'Site_343', 'Site_344', 'Site_345', 'Site_346', 'Site_347', 'Site_348',
       'Site_349', 'Site_350', 'Site_351', 'ad_unit_rank']]
y = df['CPM']
#y = df['total_revenue']


scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), columns= X.columns)

X.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train,y_train)

In [ ]:

predictions = lm.predict( X_test)
plt.scatter(y_test,predictions)
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')
# Perfect predictions
plt.plot(y_test,y_test,'r')

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score
np.sqrt(mean_squared_error(y_test,predictions))

In [ ]:
df['CPM'].mean()

APPROACH 1 : THE MODEL PERFORMS POORLY

Approach 2 : Simple linear regression where the revenue is not 0 Takling this approach because i feel maybe because the ads were visible and still the revenue was 0 maybe because the data was not recorded for the same.

In [ ]:
#trying without value where rev is 0
df2 = df[df['total_revenue'] !=0.0 ]

In [ ]:
len(df2)

In [ ]:
from sklearn.model_selection import train_test_split
X = df2[['View/measurable','measurable_impressions', 'viewable_impressions','ad_type_id','Monetization_0', 'Monetization_1', 'Advert_0', 'Advert_1', 'Advert_2', 'Advert_3', 'Advert_4', 'Advert_5',
       'Advert_6', 'Advert_7', 'Advert_8', 'Advert_9', 'Devise_0', 'Devise_1', 'geo_rank', 'Site_343', 'Site_344', 'Site_345', 'Site_346', 'Site_347', 'Site_348',
       'Site_349', 'Site_350', 'Site_351', 'ad_unit_rank']]
y = df2['CPM']
#y = df['total_revenue']


scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), columns= X.columns)

X.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train,y_train)

In [ ]:

predictions = lm.predict( X_test)
plt.scatter(y_test,predictions)
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')
# Perfect predictions
plt.plot(y_test,y_test,'r')

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score
np.sqrt(mean_squared_error(y_test,predictions))

In [ ]:
df2['CPM'].mean()

APPROACH 2: AGAIN THE MODEL PERFORMS POORLY

Approach 3 : Neural network approach with a simple dense network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

In [ ]:
X.shape

In [ ]:
#tensor flow
model = Sequential()

model.add(Dense(29,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam',loss='mse')


In [ ]:
model.fit(x=X_train.values,y=y_train.values,
         validation_data=(X_test,y_test.values),
         batch_size=128,epochs=400)

In [ ]:
predictions = model.predict(X_test)
mean_absolute_error(y_test,predictions)


In [ ]:
np.sqrt(mean_squared_error(y_test,predictions))

In [ ]:
# Our predictions
plt.scatter(y_test,predictions)

# Perfect predictions
plt.plot(y_test,y_test,'r')

approach 3 : THE MODEL PERFORMS POORLY

Approach 4: ANN with revenue not = 0

In [ ]:
#trying without value where rev is 0
df2 = df[df['total_revenue'] !=0.0 ]

In [ ]:
from sklearn.model_selection import train_test_split
X = df2[['View/measurable','measurable_impressions', 'viewable_impressions','ad_type_id','Monetization_0', 'Monetization_1', 'Advert_0', 'Advert_1', 'Advert_2', 'Advert_3', 'Advert_4', 'Advert_5',
       'Advert_6', 'Advert_7', 'Advert_8', 'Advert_9', 'Devise_0', 'Devise_1', 'geo_rank', 'Site_343', 'Site_344', 'Site_345', 'Site_346', 'Site_347', 'Site_348',
       'Site_349', 'Site_350', 'Site_351', 'ad_unit_rank']]
y = df2['CPM']
#y = df['total_revenue']


scaler = StandardScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), columns= X.columns)

X.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [ ]:
X.shape

In [ ]:
#tensor flow
model = Sequential()

model.add(Dense(29,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam',loss='mse')


In [ ]:
model.fit(x=X_train.values,y=y_train.values,
         validation_data=(X_test,y_test.values),
         batch_size=128,epochs=400)

In [ ]:
predictions = model.predict(X_test)
mean_absolute_error(y_test,predictions)


In [ ]:
np.sqrt(mean_squared_error(y_test,predictions))

In [ ]:
df2['CPM'].mean()

In [ ]:
# Our predictions
plt.scatter(y_test,predictions)

# Perfect predictions
plt.plot(y_test,y_test,'r')

# Predicting on a brand new Bid¶

In [ ]:
single_value = df2.drop('CPM',axis=1).iloc[0]

In [ ]:
single_value = scaler.transform(single_value[['View/measurable','measurable_impressions', 'viewable_impressions','ad_type_id','Monetization_0', 'Monetization_1', 'Advert_0', 'Advert_1', 'Advert_2', 'Advert_3', 'Advert_4', 'Advert_5',
       'Advert_6', 'Advert_7', 'Advert_8', 'Advert_9', 'Devise_0', 'Devise_1', 'geo_rank', 'Site_343', 'Site_344', 'Site_345', 'Site_346', 'Site_347', 'Site_348',
       'Site_349', 'Site_350', 'Site_351', 'ad_unit_rank']].values.reshape(-1, 29))

In [ ]:
single_value

Predicted value

In [ ]:
model.predict(single_value)

Actual value

In [ ]:
df2['CPM'].iloc[0]

# About the models

We were not able to accurately predict the values probably because the data does not hold a lot of information, Please let me know if i can try something else on the same. I would love to improve my model. For now i was not able to extract a lot of information from the data present. Looking forward for your thoughts on how to improve the same.